In [1]:
import torch.nn as nn
import os
import pandas as pd
import numpy as np
import pickle

In [2]:
data_dir = os.path.join(os.getcwd(),"data","ml-100k")

In [3]:
data_dir

'/Users/arnav1712/Desktop/University_Stuff/SWM/RecNet/data/ml-100k'

In [4]:
data_arr = []

In [5]:
with open(os.path.join(data_dir,"u.data")) as f:
    data = f.readlines()

In [6]:
items_arr = []

In [7]:
with open(os.path.join(data_dir,"u.item"),encoding="ISO-8859-1") as f:
    items = f.readlines()

In [8]:
for it in items:
    split = str(it).split("|")[0:2]
#     print(split)
    temp = [int(split[0]),split[1]]
    items_arr.append(temp)

In [9]:
item_df = pd.DataFrame(data=items_arr,columns=["id","item"])

In [10]:
for row in data:
    data_arr.append(row.strip().split("\t"))

In [11]:
data_arr = np.array(data_arr)

In [12]:
data_df = pd.DataFrame(data=data_arr,columns=["user_id","item_id","rating","timestamp"])

In [13]:
data_df = data_df.sort_values(['user_id', 'timestamp'], ascending=[1, 1])

In [14]:
step_size=5
max_len = 30

In [15]:
data_df.head()

,user_id,item_id,rating,timestamp
59972,1,168,5,874965478
92487,1,172,5,874965478
74577,1,165,5,874965518
48214,1,156,4,874965556
15764,1,196,5,874965677


In [16]:
unique_user_id,indices,counts = np.unique(data_df['user_id'].values,
                                          return_index=True,
                                          return_counts=True)

In [17]:
def sliding_window(item_ids,step_size,max_len):
    
    for i in range(len(item_ids),0,-step_size):
        yield item_ids[max(i-max_len,0):i]

In [18]:

def return_sequences(df,indices,unique_user_id,max_len,step_size):
    
    for idx,start_index in enumerate(indices):
        if idx==len(indices)-1:
            break
        user_id=unique_user_id[idx]

        last_index = indices[idx+1]

        
        for seq in sliding_window(df['item_id'].values[start_index:last_index],step_size,max_len) :
            
            yield (user_id,seq)

In [19]:
seq_arr = []
for seq in return_sequences(data_df,indices,unique_user_id,max_len,step_size):
    seq_arr.append(seq)

In [20]:
# !rm sequences.pkl
!touch sequences2.pkl

In [21]:
with open("sequences2.pkl","wb") as f:
    pickle.dump(seq_arr,f)

In [22]:
arr = pickle.load(open("sequences2.pkl","rb"))

In [23]:
arr[:400]

[('1',
  array(['100', '154', '9', '169', '178', '87', '16', '222', '258', '266',
         '255', '272', '271', '20', '129', '221', '6', '244', '18', '270',
         '209', '32', '189', '242', '171', '111', '256', '5', '74', '102'],
        dtype=object)),
 ('1',
  array(['51', '54', '228', '44', '86', '100', '154', '9', '169', '178',
         '87', '16', '222', '258', '266', '255', '272', '271', '20', '129',
         '221', '6', '244', '18', '270', '209', '32', '189', '242', '171'],
        dtype=object)),
 ('1',
  array(['232', '63', '139', '75', '142', '51', '54', '228', '44', '86',
         '100', '154', '9', '169', '178', '87', '16', '222', '258', '266',
         '255', '272', '271', '20', '129', '221', '6', '244', '18', '270'],
        dtype=object)),
 ('1',
  array(['140', '241', '76', '78', '226', '232', '63', '139', '75', '142',
         '51', '54', '228', '44', '86', '100', '154', '9', '169', '178',
         '87', '16', '222', '258', '266', '255', '272', '271', '20', '129'],
